# Usage description

Once a member's equipment joined a project, we need to allocate observation schedule daily to each equipment. Before scheduling, we need to calculate the entire observable time range of targets in the joined projects everyday for each equipment. (Note: we only calculate the observable time range here, therefore the time for different targets might overlap.) To do this, we need the location information, target information, and a timestamp to start calculating.  
  
### The columns you need for the databases are:  
**U_have_E (user's equipment)**
- UhaveE_ID
- longitude (deg)
- latitude (deg)
- altitude (m)
- elevation limit of equipment (deg)

**Target**
- TID
- RA (longitude) (deg)
- Dec (latitude) (deg)
  
### The returned values are:
- start time
- end time

This is the observable time range for the target.  
If the returned value is nan, it means there is no possible observation this/the upcoming night.  
  
<span style="color:blue">***Do this calculation at least once a day for each equipment.***</span>

### Requirement
- python 3.5+
- numpy 1.10+
- astropy 4.2+
- astroplan 0.7

https://astroplan.readthedocs.io/en/latest/installation.html

## Create an 'Observer' = Set 'Equipment location'

In [1]:
from astroplan import Observer
import astropy.units as u

def site_information(UhaveE_ID, longitude, latitude, altitude):
    site_inf = Observer(longitude=longitude*u.deg, latitude=latitude*u.deg, elevation=altitude*u.m, name=UhaveE_ID)
    # u.deg set the unit to degree 
    return site_inf

## Set 'Target'

In [2]:
from astroplan import FixedTarget
from astropy.coordinates import SkyCoord
import astropy.units as u

def target_information(TID, ra, dec):
    target_coord = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, frame='icrs', equinox='J2000') 
    # 'icrs' and 'J2000' are coordinate system specifications, keep it
    target = FixedTarget(coord=target_coord, name=TID)
    return target

## Calculate 'Observability'

In [3]:
from astroplan import Observer
import numpy as np
from astropy.time import TimeDelta
import astropy.units as u

half_day = TimeDelta(0.5, format='jd')
## Note: Currently, no Sun observation, twilight observation allowed.

def observable_time_range(calculation_time, site_inf, target, elevation_limit):
    elevation_limit = elevation_limit*u.deg
    if site_inf.is_night(calculation_time, horizon=-18*u.deg) == False:
        t_dusk = site_inf.twilight_evening_astronomical(calculation_time, which="next")
        t_dawn = site_inf.twilight_morning_astronomical(calculation_time, which="next")
        
        if ("{0.jd}".format(t_dusk) == 'nan') and ("{0.jd}".format(t_dawn) == 'nan'):
            # Polar day, no observation avaliable.
            t_start = np.nan
            t_end = np.nan
        
        elif ("{0.jd}".format(t_dusk) != 'nan') and ("{0.jd}".format(t_dawn) == 'nan'):
            # The day before polar night, next sunrise is not within 24 hours.
            if site_inf.target_is_up(calculation_time, target) == True:
                T_rise = site_inf.target_rise_time(calculation_time, target, which="previous", horizon=elevation_limit)
                T_set = site_inf.target_set_time(calculation_time, target, which="next", horizon=elevation_limit)
                if "{0.jd}".format(T_set) != 'nan':
                    # Observe from dusk till target set.
                    t_start = t_dusk
                    t_end = T_set
                    
                elif "{0.jd}".format(T_set) == 'nan':
                    # Target never set, observation avaliable from dusk till 12 hours later (more calculations will be execute later).
                    t_start = t_dusk
                    t_end = t_start + half_day
                    
            else:
                T_rise = site_inf.target_rise_time(calculation_time, target, which="next", horizon=elevation_limit)
                T_set = site_inf.target_set_time(calculation_time, target, which="next", horizon=elevation_limit)
                if "{0.jd}".format(T_rise) != 'nan' and "{0.jd}".format(T_set) != 'nan':
                    t_end = T_set
                    if t_dusk > T_rise:
                        t_start = t_dusk
                    else:
                        t_start = T_rise
                    
                elif "{0.jd}".format(T_rise) == 'nan' and "{0.jd}".format(T_set) == 'nan':
                    t_start = np.nan
                    t_end = np.nan
                    
        elif ("{0.jd}".format(t_dusk) != 'nan') and ("{0.jd}".format(t_dawn) != 'nan'):
            if site_inf.target_is_up(calculation_time, target) == True:
                T_rise = site_inf.target_rise_time(calculation_time, target, which="previous", horizon=elevation_limit)
                T_set = site_inf.target_set_time(calculation_time, target, which="next", horizon=elevation_limit)
                if "{0.jd}".format(T_set) != 'nan':
                    if t_dusk >= T_set:
                        # Target set before dusk, no observation tonight.
                        t_start = np.nan
                        t_end = np.nan
                    else:
                        if t_dawn > T_set:
                            t_start = t_dusk
                            t_end = T_set
                        else:
                            t_start = t_dusk
                            t_end = t_dawn
                        
                elif "{0.jd}".format(T_set) == 'nan':
                    # Target never set, observation avaliable from dusk till 12 hours later (more calculations will be execute later).
                    t_start = t_dusk
                    t_end = t_start + half_day
                
            else:
                T_rise = site_inf.target_rise_time(calculation_time, target, which="next", horizon=elevation_limit)
                T_set = site_inf.target_set_time(calculation_time, target, which="next", horizon=elevation_limit)
                
                if ("{0.jd}".format(T_rise) == 'nan') and ("{0.jd}".format(T_set) == 'nan'):
                    # Target never rise.
                    t_start = np.nan
                    t_end = np.nan
                    
                elif ("{0.jd}".format(T_rise) != 'nan') and ("{0.jd}".format(T_set) != 'nan'):
                    if t_dawn <= T_rise:
                        # Target won't rise until dawn, no observation tonight.
                        t_start = np.nan
                        t_end = np.nan
                    else:
                        if t_dusk > T_rise:
                            t_start = t_dusk
                        else:
                            t_start = T_rise
        
                        if t_dawn > T_set:
                            t_end = T_set
                        else:
                            t_end = t_dawn


    elif site_inf.is_night(calculation_time, horizon=-18*u.deg) == True:
        t_dusk = site_inf.twilight_evening_astronomical(calculation_time, which="previous")
        t_dawn = site_inf.twilight_morning_astronomical(calculation_time, which="next")
        
        if ("{0.jd}".format(t_dusk) == 'nan') and ("{0.jd}".format(t_dawn) == 'nan'):
            # Polar night, can observe at any time.
            if site_inf.target_is_up(calculation_time, target) == True:
                T_rise = site_inf.target_rise_time(calculation_time, target, which="previous", horizon=elevation_limit)
                T_set = site_inf.target_set_time(calculation_time, target, which="next", horizon=elevation_limit)
                if ("{0.jd}".format(T_rise) == 'nan') and ("{0.jd}".format(T_set) == 'nan'):
                    # Target never set.
                    t_start = calculation_time
                    t_end = t_start + half_day
                    
                elif ("{0.jd}".format(T_rise) != 'nan') and ("{0.jd}".format(T_set) != 'nan'):
                    t_start = calculation_time
                    t_end = T_set
                    
            else:
                T_rise = site_inf.target_rise_time(calculation_time, target, which="next", horizon=elevation_limit)
                T_set = site_inf.target_set_time(calculation_time, target, which="next", horizon=elevation_limit)
                if ("{0.jd}".format(T_rise) == 'nan') and ("{0.jd}".format(T_set) == 'nan'):
                    # Target never rise.
                    t_start = np.nan
                    t_end = np.nan
                    
                elif ("{0.jd}".format(T_rise) != 'nan') and ("{0.jd}".format(T_set) != 'nan'):
                    t_start = T_rise
                    t_end = T_set
    
        elif ("{0.jd}".format(t_dusk) == 'nan') and ("{0.jd}".format(t_dawn) != 'nan'):
            # The last polar night, Sun will rise the next day.
            if site_inf.target_is_up(calculation_time, target) == True:
                T_rise = site_inf.target_rise_time(calculation_time, target, which="previous", horizon=elevation_limit)
                T_set = site_inf.target_set_time(calculation_time, target, which="next", horizon=elevation_limit)
                if ("{0.jd}".format(T_rise) == 'nan') and ("{0.jd}".format(T_set) == 'nan'):
                    # Target never set.
                    t_start = calculation_time
                    t_end = t_dawn
                    
                elif ("{0.jd}".format(T_rise) != 'nan') and ("{0.jd}".format(T_set) != 'nan'):
                    t_start = calculation_time
                    if t_dawn > T_set:
                        t_end = T_set
                    else:
                        t_end = t_dawn
                        
            else:
                T_rise = site_inf.target_rise_time(calculation_time, target, which="next", horizon=elevation_limit)
                T_set = site_inf.target_set_time(calculation_time, target, which="next", horizon=elevation_limit)
                if ("{0.jd}".format(T_rise) == 'nan') and ("{0.jd}".format(T_set) == 'nan'):
                    # Target never rise.
                    t_start = np.nan
                    t_end = np.nan
                    
                elif ("{0.jd}".format(T_rise) != 'nan') and ("{0.jd}".format(T_set) != 'nan'):
                    if t_dawn <= T_rise:
                        # Target won't rise until dawn, no observation tonight.
                        t_start = np.nan
                        t_end = np.nan
                    else:
                        t_start = T_rise
                        if t_dawn > T_set:
                            t_end = T_set
                        else:
                            t_end = t_dawn         
           
        elif ("{0.jd}".format(t_dusk) != 'nan') and ("{0.jd}".format(t_dawn) != 'nan'):
            if site_inf.target_is_up(calculation_time, target) == True:
                T_rise = site_inf.target_rise_time(calculation_time, target, which="previous", horizon=elevation_limit)
                T_set = site_inf.target_set_time(calculation_time, target, which="next", horizon=elevation_limit)
                if ("{0.jd}".format(T_rise) == 'nan') and ("{0.jd}".format(T_set) == 'nan'):
                    # Target never set.
                    t_start = calculation_time
                    t_end = t_dawn
                    
                elif ("{0.jd}".format(T_rise) != 'nan') and ("{0.jd}".format(T_set) != 'nan'):
                    t_start = calculation_time
                    if t_dawn > T_set:
                        t_end = T_set
                    else:
                        t_end = t_dawn
            
            else:
                T_rise = site_inf.target_rise_time(calculation_time, target, which="next", horizon=elevation_limit)
                T_set = site_inf.target_set_time(calculation_time, target, which="next", horizon=elevation_limit)
                if ("{0.jd}".format(T_rise) == 'nan') and ("{0.jd}".format(T_set) == 'nan'):
                    # Target never rise.
                    t_start = np.nan
                    t_end = np.nan
                    
                elif ("{0.jd}".format(T_rise) != 'nan') and ("{0.jd}".format(T_set) != 'nan'):
                    if t_dawn <= T_rise:
                        # Target won't rise until dawn, no observation tonight.
                        t_start = np.nan
                        t_end = np.nan
                    else:
                        t_start = T_rise
                        if t_dawn > T_set:
                            t_end = T_set
                        else:
                            t_end = t_dawn
    # If t_start or t_end is not given a value, jump warning.
    
    return t_start, t_end


## Main program

# Equipment Schedule Arrangements

In [9]:
from datetime import datetime
from astropy.time import Time
import time

import datetime as dt
import astral
from datetime import timedelta
import pandas as pd

# Given arguments. Import this from your databases!!!

# Equipment information
UhaveE_ID = 15
latitude = -37.64734822538928
longitude = -21.6581509963778
altitude = 256.84036136517295
elevation_limit = 26.025507509843457
time_zone = "UTC-1"

# Target Informaiton
TID = 3986
ra = 297.4286849321643
dec = +44.0998098091643


def interval(value):
    
    if value<6:
        return 1
    elif value<12:
        return 2
    elif value<18:
        return 3    
    elif value<24:
        return 4  

def clean(value):
    if "+" in value:
        value = value.split("+")[0]
    if "T" in value:
        value = " ".join((value.split("T")))
    if "." in value:
        value = value.split(".")[0]
    return value
    
if __name__ == '__main__':
    
    calculation_time = datetime.now()
    print("present : "+clean(str(calculation_time)))
    print("\n")
    
    location_XiChong = astral.Location(('', '',  latitude,longitude, "Europe/London", 0))
    sunrise=location_XiChong.sunrise(date=calculation_time,local=False)
    print(sunrise)
    sunrise += timedelta(hours=int(time_zone[3:]))
    sunrise_new=clean(str(sunrise))
    print("today sunrise : "+clean(sunrise_new))

    sunset=location_XiChong.sunset(date=calculation_time,local=False)
    sunset += timedelta(hours=int(time_zone[3:]))
    sunset_new=clean(str(sunset))
    print("today sunset  : "+clean(sunset_new)) 
    
    print("\n")
    

    calculation_time2 = calculation_time + timedelta(days=+1)
    print("tomorrow : "+clean(str(calculation_time2)))
    print("\n")

    location_XiChong = astral.Location(('', '',  latitude,longitude, "Europe/London", 0))
    tom_sunrise=location_XiChong.sunrise(date=calculation_time2,local=False)
    print(tom_sunrise)
    tom_sunrise += timedelta(hours=int(time_zone[3:]))
    tom_sunrise_new=clean(str(tom_sunrise))
    print("tomorrow sunrise : "+tom_sunrise_new)

    tom_sunset=location_XiChong.sunset(date=calculation_time2,local=False)
    tom_sunset += timedelta(hours=int(time_zone[3:]))
    tom_sunset_new=clean(str(tom_sunset))
    print("tomorrow sunset  : "+tom_sunset_new) 
    
    
    site_inf = site_information(UhaveE_ID, longitude, latitude, altitude)
    target = target_information(TID, ra, dec)
    
    
    date = calculation_time.day
    
    part = interval(calculation_time.hour)
    
    
   
    calculation_time = datetime.fromisoformat(str(calculation_time))
    calculation_time = site_inf.datetime_to_astropy_time(calculation_time)
    calculation_time = Time(calculation_time, format='fits', scale='utc')

    t_start, t_end = observable_time_range(calculation_time, site_inf, target, elevation_limit)
    
    try:
        t_start.format = 'fits'
    except AttributeError:
        pass
    try:
        t_end.format = 'fits'
    except AttributeError:
        pass
    
    print("\n")
    print('start observation: %s \nend observation %s' % (clean(str(t_start)), clean(str(t_end))))
    print("\n")
    
    
    if pd.isnull(t_start):
        print("not shown")
      #  print("yes!!!!!")
    else:
        if t_start >= sunrise and t_start <= sunset:
            t_start = sunset
        elif t_start >= tom_sunrise and t_start <= tom_sunset:
            t_start = tom_sunset
            
        if t_end >= sunrise and t_end <= sunset:
            t_end = sunrise
        elif t_end >= tom_sunrise and t_start <= tom_sunset:
            t_end = tom_sunrise

        if sunrise >= t_start and sunset <= t_end:
            t_start = sunset

        if tom_sunrise >= t_start and tom_sunset <= t_end:
            t_end = tom_sunrise
            
        if t_end == t_start or t_start >= t_end:
            print("not shown")
            print('start observation: %s \nend observation %s' % (clean(str(t_start)), clean(str(t_end))))
        else:
            if int(clean(str(t_start)).split()[0].split("-")[-1]) == date and interval(int(clean(str(t_start)).split()[1].split(":")[0])) == part + 1 :
                print("shown")
                print('start observation: %s \nend observation %s' % (clean(str(t_start)), clean(str(t_end))))
            elif int(clean(str(t_start)).split()[0].split("-")[-1]) != date and part == 4 and interval(int(clean(str(t_start)).split()[1].split(":")[0])) ==1:
                print("shown")
                print('start observation: %s \nend observation %s' % (clean(str(t_start)), clean(str(t_end))))
            else:
                print("not shown")
                print('start observation: %s \nend observation %s' % (clean(str(t_start)), clean(str(t_end))))           


present : 2021-01-19 12:41:35


2021-01-19 06:26:42+00:00
today sunrise : 2021-01-19 05:26:42
today sunset  : 2021-01-19 19:48:09


tomorrow : 2021-01-20 12:41:35


2021-01-20 06:27:48+00:00
tomorrow sunrise : 2021-01-20 05:27:48
tomorrow sunset  : 2021-01-20 19:47:38


start observation: 2021-01-19 22:34:47 
end observation 2021-01-20 10:34:47


shown
start observation: 2021-01-19 22:34:47 
end observation 2021-01-20 05:27:48


# Importing TSV files

In [24]:
import pandas as pd

equipment = pd.read_csv("dataset/Equipment_db.tsv",sep="\t")
equipment.columns = [i.replace(" ","_") for i in equipment.columns]
#equipment.columns = [i.replace(" ","").replace("(","").replace(")","").replace("/","").replace(",","") for i in equipment.columns]

In [25]:
equipment

,EID,aperture,FoV,"pixel_scale_(""/pix)","tracking_accuracy_("")",limiting_magnitude_(mag),elevation_limit_(deg),"mount_type_(german,_fork,_theodolite)","camera_type_(colored,_mono)","camera_type_(cooled,_uncooled)",Johnson_B,Johnson_V,Johnson_R,SDSS_u,SDSS_g,SDSS_r,SDSS_i,SDSS_z
0,26,3.368304,226.616457,2.793210,0.839967,10.367366,21.900718,german,mono,cooled,y,y,y,y,n,y,y,y
1,27,9.467561,478.882647,0.189069,0.187561,14.009303,24.874248,fork,mono,cooled,y,y,y,y,n,y,y,y
2,28,1.360916,41.052733,2.049245,0.685276,13.816671,29.379780,german,mono,uncooled,y,n,n,y,y,y,y,y
3,29,9.618051,302.878379,1.797798,0.325474,14.327599,34.874938,fork,colored,cooled,y,y,y,n,y,y,y,n
4,30,1.510077,75.866117,0.634982,0.505984,8.451051,31.795557,fork,mono,cooled,y,y,n,y,y,y,n,n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,1995,1.054357,515.138004,0.927321,0.574970,15.212146,29.600526,german,colored,cooled,n,y,y,y,y,y,y,n
1970,1996,8.015035,121.936745,0.533497,0.296749,13.749275,18.319929,fork,colored,cooled,n,y,y,y,n,n,y,y
1971,1997,3.982763,27.055468,1.371728,0.652889,10.177209,22.222588,fork,colored,cooled,y,y,y,y,y,y,n,y
1972,1998,9.267684,412.199020,0.770935,0.925310,8.676038,33.165160,fork,colored,cooled,y,y,n,y,y,y,y,y


In [33]:
projects = pd.read_csv("dataset/Project_db.tsv",sep="\t")
projects.columns = [i.replace(" ","_") for i in projects.columns]
#projects.columns = [i.replace(" ","").replace("(","").replace(")","").replace("/","").replace(",","") for i in projects.columns]

In [34]:
projects

,PID,title,project_type,PI,description,aperture_upper_limit,aperture_lower_limit,FoV_upper_limit,FoV_lower_limit,pixel_scale_upper_limit_(/pix),...,"camera_type_(colored,mono)","camera_type(cooled,uncooled)",Johnson_B,Johnson_V,Johnson_R,SDSS_u,SDSS_g,SDSS_r,SDSS_i,SDSS_z
0,0,Observe Supernova 1,transient,617,Observe Supernova 1 76 times,30,30,600,1,3,...,mono,cooled,y,y,y,n,y,y,y,y
1,1,Observe GRB 1,regular,641,Observe GRB 1 45 times,30,30,600,1,3,...,mono,cooled,y,y,y,y,y,y,y,y
2,2,Observe Exoplanets 1,regular,25,Observe Exoplanets 1 65 times,30,30,600,1,3,...,colored,cooled,y,y,y,y,y,y,n,y
3,3,Observe Exoplanets 2,transient,140,Observe Exoplanets 2 45 times,30,30,600,1,3,...,colored,cooled,y,n,y,y,y,n,y,y
4,4,Observe Supernova 2,regular,737,Observe Supernova 2 90 times,30,30,600,1,3,...,colored,cooled,y,n,y,y,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,Observe Exoplanets 305,transient,266,Observe Exoplanets 305 99 times,30,30,600,1,3,...,mono,cooled,y,n,y,y,y,y,n,y
996,996,Observe GRB 103,transient,406,Observe GRB 103 35 times,30,30,600,1,3,...,colored,cooled,y,y,y,y,y,y,y,y
997,997,Observe Galaxy 140,regular,72,Observe Galaxy 140 82 times,30,30,600,1,3,...,mono,cooled,n,n,y,n,y,y,y,y
998,998,Observe Exoplanets 306,regular,503,Observe Exoplanets 306 63 times,30,30,600,1,3,...,colored,uncooled,y,y,n,y,y,n,y,y


In [10]:
user = pd.read_csv("dataset/User_db.tsv",sep="\t")
user.columns = [i.replace(" ","_") for i in user.columns]

In [11]:
user

,UID,username,name,email,affiliation,title,country,Interest,Type
0,0,xdkjmiqi43,Mveabjz,Mveabjz@Observatory0.com,Observatory0,Amature,Taiwan,5904 5387 4156 554 8368 9205 8676 893 5882 193...,regular
1,1,yhdnunvu47,Iwqilau,Iwqilau@Observatory1.com,Observatory1,Student,Taiwan,6842 8401 3900 6005 2118 7165 9831 8386 6600 7...,regular
2,2,zqlpskqs0,Uutajuh,Uutajuh@Observatory2.com,Observatory2,Student,Taiwan,8036 4715 803 3624 8279 3763 3400 4646 4919 91...,transient
3,3,hxsutzhs57,Gibuzvz,Gibuzvz@Observatory3.com,Observatory3,Amature,Taiwan,3116 4821 8627 9761 989 5639 8589 9346 3852 17...,transient
4,4,blvwhidb47,Mdpwgdb,Mdpwgdb@Observatory4.com,Observatory4,Postdoc,Japan,8553 5088 5744 1470 3453 7385 6483 6603 4225 2...,transient
...,...,...,...,...,...,...,...,...,...
995,995,vbidgqnr11,Ygxaruc,Ygxaruc@Observatory995.com,Observatory995,Student,Taiwan,5678 6971 766 1427 8500 5951 2369 1227 1960 73...,regular
996,996,ketmpzdf36,Qplgtfs,Qplgtfs@Observatory996.com,Observatory996,Student,Japan,4615 8334 6229 9255 5233 9446 9866 6197 6368 7...,regular
997,997,ggygywni2,Dgxzsxn,Dgxzsxn@Observatory997.com,Observatory997,Amature,Spain,9311 3332 909 6582 9957 5196 3373 8522 2866 79...,regular
998,998,epxhhlta41,Rashybv,Rashybv@Observatory998.com,Observatory998,Student,Taiwan,9575 1292 8944 4019 5072 8803 1168 8268 5943 9...,transient


In [35]:
project_targets = pd.read_csv("dataset/Project's_Target_db.tsv",sep="\t")
project_targets.columns = [i.replace(" ","_") for i in project_targets.columns]
#project_targets.columns = [i.replace(" ","").replace("(","").replace(")","").replace("/","").replace(",","") for i in project_targets.columns]

In [36]:
project_targets

,PhaveT_ID,PID,TID,Johnson_B,Johnson_V,Johnson_R,SDSS_u,SDSS_g,SDSS_r,SDSS_i,SDSS_z
0,0,0,126,y,y,y,n,y,y,y,y
1,1,0,246,n,y,y,n,y,n,n,y
2,2,0,271,n,y,y,n,y,n,n,y
3,3,0,272,n,y,y,n,y,n,n,n
4,4,0,282,n,y,n,n,y,n,n,n
...,...,...,...,...,...,...,...,...,...,...,...
103935,103935,999,9307,n,n,n,n,n,n,n,n
103936,103936,999,9564,n,n,n,n,n,n,n,n
103937,103937,999,9646,n,n,n,n,n,n,n,n
103938,103938,999,9748,n,n,n,n,n,n,n,n


In [37]:
user_equipment = pd.read_csv("dataset/User's_Equipment_db.tsv",sep="\t")
user_equipment.columns = [i.replace(" ","_") for i in user_equipment.columns]

In [38]:
user_equipment

,UhaveE_ID,UID,EID,site,longitude,latitude,altitude,time_zone,daylight_saving,water_vapor,light_pollution(m/sas)
0,0,0,290,Mauna Kea Observatory,-149.165169,-34.270023,3776.692422,UTC-9,n,4.306448,21.429877
1,1,0,520,Large Millimeter Telescope,-0.427690,-37.447115,1482.029639,UTC+0,n,5.497076,20.136897
2,2,1,2539,Gornergrat Observatory,-44.267769,-64.184052,175.080140,UTC-2,n,4.093031,17.271445
3,3,1,2355,Large Millimeter Telescope,-0.789974,-58.779978,4338.562097,UTC+0,y,9.975761,17.214697
4,4,2,186,Plateau Observatory,-114.195111,61.244718,3555.606604,UTC-7,n,3.660963,19.962710
...,...,...,...,...,...,...,...,...,...,...,...
2995,2995,998,2380,Atacama Cosmology Telescope,-47.346840,34.522964,1919.103574,UTC-3,y,9.513648,18.144141
2996,2996,998,1247,Plateau Observatory,167.334848,-69.556104,203.983734,UTC+11,n,4.731400,22.154028
2997,2997,999,1611,University of Tokyo Atacama Observatory,-50.547928,39.498697,5047.069629,UTC-3,n,8.341596,19.841670
2998,2998,999,1624,Chacaltaya Astrophysical Observatory,107.629736,20.727395,55.892595,UTC+7,n,4.522002,23.585649


# (Bonus) Recommendations of Projects

In [185]:
import csv
    
total = []
for eid in user_equipment[user_equipment.UID == 5].EID.tolist():
    print(eid)

    if eid > 1999:
        eid -= 2000 

    print(eid)

    candidate = []

    for pid in projects.PID.values:
       # if (equipment[equipment.EID == eid]['aperture'].values[0] > projects[projects.PID==pid]['aperture_upper_limit'].values[0]):
       #     continue
       # if (equipment[equipment.EID == eid]['aperture'].values[0] < projects[projects.PID==pid]['aperture_lower_limit'].values[0]):
       #     continue    
        if (equipment[equipment.EID == eid]['FoV'].values[0] > projects[projects.PID==pid]['FoV_upper_limit'].values[0]):
            continue
        if (equipment[equipment.EID == eid]['FoV'].values[0] < projects[projects.PID==pid]['FoV_lower_limit'].values[0]):
            continue  
        if (equipment[equipment.EID == eid]['pixel_scale_("/pix)'].values[0] > projects[projects.PID==pid]['pixel_scale_upper_limit_(/pix)'].values[0]):
            continue
        if (equipment[equipment.EID == eid]['pixel_scale_("/pix)'].values[0] < projects[projects.PID==pid]['pixel_scale_lower_limit_(/pix)'].values[0]):
            continue
        if (equipment[equipment.EID == eid]['mount_type_(german,_fork,_theodolite)'].values[0] != projects[projects.PID==pid]['mount_type(german,fork,theodolite)'].values[0]):
            continue
        if (equipment[equipment.EID == eid]['camera_type_(cooled,_uncooled)'].values[0] != projects[projects.PID==pid]['camera_type(cooled,uncooled)'].values[0]):
            continue
        if equipment[equipment.EID == eid]['Johnson_B'].values[0] == "n" and projects[projects.PID==pid]['Johnson_B'].values[0] == "y":
            continue
        if equipment[equipment.EID == eid]['Johnson_V'].values[0] == "n" and projects[projects.PID==pid]['Johnson_V'].values[0] == "y":
            continue
        if equipment[equipment.EID == eid]['Johnson_R'].values[0] == "n" and projects[projects.PID==pid]['Johnson_R'].values[0] == "y":
            continue
        if equipment[equipment.EID == eid]['SDSS_u'].values[0] == "n" and projects[projects.PID==pid]['SDSS_u'].values[0] == "y":
            continue
        if equipment[equipment.EID == eid]['SDSS_g'].values[0] == "n" and projects[projects.PID==pid]['SDSS_g'].values[0] == "y":
            continue
        if equipment[equipment.EID == eid]['SDSS_r'].values[0] == "n" and projects[projects.PID==pid]['SDSS_r'].values[0] == "y":
            continue
        if equipment[equipment.EID == eid]['SDSS_i'].values[0] == "n" and projects[projects.PID==pid]['SDSS_i'].values[0] == "y":
            continue
        if equipment[equipment.EID == eid]['SDSS_z'].values[0] == "n" and projects[projects.PID==pid]['SDSS_z'].values[0] == "y":
            continue


        candidate.append(pid)

    print(candidate)

    tagets_list = []    
    ability = equipment[equipment.EID == eid].values.tolist()[0][-8:]
    interest = [int(i) for i in user[user["UID"] == 5]['Interest'].values[0].split()]
    type_of_project = user[user["UID"] == 5]["Type"].values[0]

    for pid in candidate:

        for tid in project_targets[project_targets.PID == pid].values:  
            if tid[2] not in interest:
                continue

            scoring = 0
            ability2 = tid[3:]
            for i in range(8):
                if ability2[i] == "y" and ability[i] == "n":
                    break

                if i == 7:
                    if type_of_project == projects[projects.PID==pid]['project_type'].values[0]: 
                        scoring += 50
                    if tid[2] in interest:
                        scoring += 100  
                    tagets_list.append([projects[projects.PID==pid].title.values[0],targets[targets.TID==tid[2]].Name.values[0],user[user.UID==projects[projects.PID==pid].PI.values[0]].username.values[0],user[user.UID==projects[projects.PID==pid].PI.values[0]].email.values[0],scoring])
    tagets_list = sorted(tagets_list, key=lambda s: s[-1], reverse=True)


    with open('recommend_project_for_EID-{}.csv'.format(str(eid)), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        writer.writerow(['Project Name', 'Target Name', 'Project Manager', "Email"])

        for i in tagets_list:
            print(i)
            writer.writerow(i[:-1])

    total.append(tagets_list)

print("\n")
print("sorting consequence")
print("\n")
final_targets = dict()

for i in total:
    for j in i:
        if j[0]+" "+j[1] not in final_targets:
            final_targets[j[0]+" "+j[1]] = j[-1]

        else:
            final_targets[j[0]+" "+j[1]] += j[-1]

done = [i for i in final_targets]
done = sorted(done, key=lambda s: final_targets[s], reverse=True)

for i in done:
    print(i+" "+str(final_targets[i]))

51
51
[3, 36, 47, 86, 89, 98, 102, 150, 159, 167, 173, 176, 194, 222, 244, 258, 295, 333, 342, 369, 395, 398, 407, 417, 436, 445, 449, 453, 473, 494, 514, 523, 525, 531, 537, 585, 598, 601, 608, 610, 616, 621, 626, 627, 639, 641, 654, 682, 687, 704, 710, 748, 750, 757, 774, 776, 782, 794, 817, 821, 827, 860, 885, 907, 914, 918, 940, 960, 985, 987]
['Observe Star 19', 'V* V1043 Her                       ', 'zewjyemq31', 'Akrevix@Observatory841.com', 150]
['Observe Galaxy 27', 'IRAS 21172+4944                    ', 'kxxmupxf56', 'Azkinse@Observatory194.com', 150]
['Observe Exoplanets 61', 'IRAS 19481+4358                    ', 'kugwxedm24', 'Mmrifef@Observatory134.com', 150]
['Observe Exoplanets 187', 'V* V1190 Cyg                       ', 'mazgnnpl13', 'Gfmzfcr@Observatory989.com', 150]
['Observe Galaxy 108', 'V* V1190 Cyg                       ', 'fwttjmcg40', 'Todgdtl@Observatory423.com', 150]
['Observe Exoplanets 254', 'ROTSE1 J175137.96+492317.8         ', 'qzkfmarm40', 'Kspcyxs@Obs

# (Bonus) Recommendations of Users

In [197]:
def convert(title):
    if title == 'Amature':
        return 1
    if title == 'Student':
        return 2
    if title == 'Postdoc':
        return 3
    if title == 'Professor':
        return 4
    
    
friends_list = []
UID = 5

for uid in user.UID.tolist():
    if uid == UID:
        continue
        
    scoring = 0   

#    if user[user["UID"]==5].title.values[0] == user[user["UID"]==uid].title.values[0]:
 #       scoring += 100
  
    scoring += (300 - 100*abs((convert(user[user["UID"]==UID].title.values[0])-convert(user[user["UID"]==uid].title.values[0]))))

    if user[user["UID"]==UID].country.values[0] == user[user["UID"]==uid].country.values[0]:
        scoring += 100
    
    scoring += 200 * len(set([i for i in user[user["UID"]==UID].Interest.values[0].split()])&set([i for i in user[user["UID"]==uid].Interest.values[0].split()]))
    
    friends_list.append([user[user["UID"]==uid].username.values[0],user[user["UID"]==uid].country.values[0],user[user["UID"]==uid].title.values[0],user[user["UID"]==uid].email.values[0],set([i for i in user[user["UID"]==UID].Interest.values[0].split()])&set([i for i in user[user["UID"]==uid].Interest.values[0].split()]),scoring])
   
    
friends_list = sorted(friends_list, key=lambda s: s[-1], reverse=True)


with open('recommend_friend_for_UID-{}.csv'.format(str(UID)), 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    writer.writerow(['username', 'Country', 'Title', "Email","Common interest target"])
    times =0 
    for i in friends_list:
        print(i)
        writer.writerow([i[0],i[1],i[2],i[3]," && ".join([targets[targets.TID==int(a)].Name.values[0].strip() for a in list(i[4])])])
        times += 1
        if times ==19:
            break


['vnhmudso43', 'Taiwan', 'Student', 'Amlkqrc@Observatory207.com', {'2493'}, 600]
['baqiyndi1', 'Taiwan', 'Student', 'Xybkqid@Observatory312.com', {'8434'}, 600]
['ajoluzes54', 'Taiwan', 'Student', 'Yjilhsc@Observatory359.com', {'3164'}, 600]
['dzebzzbp54', 'Taiwan', 'Student', 'Owtbyog@Observatory366.com', {'6502'}, 600]
['czlfizwa53', 'Japan', 'Amature', 'Mbcnhuz@Observatory380.com', {'773', '7458'}, 600]
['efharsvf14', 'Taiwan', 'Student', 'Tsamvwc@Observatory565.com', {'1046'}, 600]
['vgacexxk52', 'Taiwan', 'Student', 'Xbmqsjx@Observatory686.com', {'3493'}, 600]
['fozlejjb23', 'Taiwan', 'Student', 'Wltbgzh@Observatory943.com', {'9522'}, 600]
['zkewepnq16', 'Taiwan', 'Amature', 'Zemqhoe@Observatory31.com', {'8434'}, 500]
['setiihbo39', 'Korea', 'Student', 'Hulfgex@Observatory91.com', {'3164'}, 500]
['yspkikwr29', 'Taiwan', 'Amature', 'Wvuhifc@Observatory205.com', {'8529'}, 500]
['ekyfxfyr28', 'Japan', 'Student', 'Tlugwvm@Observatory394.com', {'8303'}, 500]
['ondpdixn29', 'Taiwan', 'A